<a href="https://colab.research.google.com/github/Moorea-AI/ANALITICA3/blob/main/b_preprocesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [569]:
#### Cargar paquetes
import pandas as pd ### para manejo de datos
import sqlite3 as sql #### para bases de datos sql
import sys ## saber ruta de la que carga paquetes


In [570]:
#conectamos drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [571]:
sys.path.append('/content/drive/MyDrive')

In [572]:
## conectamos las funciones
import a_funciones as funciones

In [573]:
#se generan las url para conectar las bases de datos desde git hub
employee = 'https://raw.githubusercontent.com/Moorea-AI/ANALITICA3/main/databases/employee_survey_data.csv'
general = 'https://raw.githubusercontent.com/Moorea-AI/ANALITICA3/main/databases/general_data.csv'
manager = 'https://raw.githubusercontent.com/Moorea-AI/ANALITICA3/main/databases/manager_survey.csv'
retirement = 'https://raw.githubusercontent.com/Moorea-AI/ANALITICA3/main/databases/retirement_info.csv'

In [574]:
# se leen los archivos csv
df_employee=pd.read_csv(employee)
df_general=pd.read_csv(general)
df_manager=pd.read_csv(manager)
df_retirement=pd.read_csv(retirement)

In [575]:
### por facilidad se convierten los nombres de las columnas a minuscula
df_employee = df_employee.rename(columns=lambda x: x.lower())
df_general = df_general.rename(columns=lambda x: x.lower())
df_manager = df_manager.rename(columns=lambda x: x.lower())
df_retirement = df_retirement.rename(columns=lambda x: x.lower())

In [576]:
print(df_employee.shape)#se consulta el tamaño del DF
print(df_general.shape)#se consulta el tamaño del DF
print(df_manager.shape)#se consulta el tamaño del DF
print(df_retirement.shape)#se consulta el tamaño del DF

(8820, 6)
(8820, 25)
(8820, 5)
(711, 7)


In [577]:
# resumen de información de las tablas
df_employee.info()
df_general.info()
df_manager.info()
df_retirement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8820 entries, 0 to 8819
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   unnamed: 0               8820 non-null   int64  
 1   employeeid               8820 non-null   int64  
 2   environmentsatisfaction  8770 non-null   float64
 3   jobsatisfaction          8780 non-null   float64
 4   worklifebalance          8744 non-null   float64
 5   datesurvey               8820 non-null   object 
dtypes: float64(3), int64(2), object(1)
memory usage: 413.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8820 entries, 0 to 8819
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   unnamed: 0               8820 non-null   int64  
 1   age                      8820 non-null   int64  
 2   businesstravel           8820 non-null   object 
 3   department      

In [578]:
 ###consultar el número de nulos
print(df_employee.isnull().sum())
print(df_general.isnull().sum())
print(df_manager.isnull().sum())
print(df_retirement.isnull().sum())

unnamed: 0                  0
employeeid                  0
environmentsatisfaction    50
jobsatisfaction            40
worklifebalance            76
datesurvey                  0
dtype: int64
unnamed: 0                  0
age                         0
businesstravel              0
department                  0
distancefromhome            0
education                   0
educationfield              0
employeecount               0
employeeid                  0
gender                      0
joblevel                    0
jobrole                     0
maritalstatus               0
monthlyincome               0
numcompaniesworked         38
over18                      0
percentsalaryhike           0
standardhours               0
stockoptionlevel            0
totalworkingyears          18
trainingtimeslastyear       0
yearsatcompany              0
yearssincelastpromotion     0
yearswithcurrmanager        0
infodate                    0
dtype: int64
unnamed: 0           0
employeeid           

Las variables que tienen datos nulos son:




1. environmentsatisfaction    50
2. jobsatisfaction            40
3. worklifebalance            76
4. numcompaniesworked         38
5. totalworkingyears          18
6. resignationreason          70

In [579]:
#### Convertir campos a formato fecha
df_employee["datesurvey"]=pd.to_datetime(df_employee['datesurvey'], format="%Y/%m/%d")
df_general["infodate"]=pd.to_datetime(df_general['infodate'], format="%Y/%m/%d")
df_manager["surveydate"]=pd.to_datetime(df_manager['surveydate'], format="%Y/%m/%d")
df_retirement["retirementdate"]=pd.to_datetime(df_retirement['retirementdate'], format="%Y/%m/%d")

In [580]:
## eliminar columnas
df_employee =df_employee.drop(["unnamed: 0"], axis=1)
df_general =df_general.drop(["unnamed: 0","over18", "employeecount","standardhours"], axis=1)
df_manager =df_manager.drop(["unnamed: 0"], axis=1)
df_retirement =df_retirement.drop(["unnamed: 0","unnamed: 0.1"], axis=1)

In [581]:
df_employee.head(5)

,employeeid,environmentsatisfaction,jobsatisfaction,worklifebalance,datesurvey
0,1,3.0,4.0,2.0,2015-12-31
1,2,3.0,2.0,4.0,2015-12-31
2,3,2.0,2.0,1.0,2015-12-31
3,4,4.0,4.0,3.0,2015-12-31
4,5,4.0,1.0,3.0,2015-12-31


In [582]:
df_manager.head(5)

,employeeid,jobinvolvement,performancerating,surveydate
0,1,3,3,2015-12-31
1,2,2,4,2015-12-31
2,3,3,3,2015-12-31
3,4,2,3,2015-12-31
4,5,3,3,2015-12-31


In [583]:
#crear base de datos en SQL
conn= sql.connect("data\\db_empleados") ### crea una base de datos con el nombre dentro de comillas, si existe crea una conexión.
cur=conn.cursor() ### ejecutar funciones  en BD


In [584]:
### Llevar tablas a base de datos
df_employee.to_sql("employee",conn,if_exists="replace")
df_general.to_sql("general",conn,if_exists="replace")
df_manager.to_sql("manager",conn,if_exists="replace")
df_retirement.to_sql("retirement",conn,if_exists="replace")

711

In [585]:
cur.execute("Select name from sqlite_master where type='table'") ### consultar bases de datos
cur.fetchall()

[('BD',), ('employee',), ('general',), ('manager',), ('retirement',)]

In [586]:
BD = pd.read_sql("""
SELECT
    e.employeeid,
    e.environmentsatisfaction,
    e.jobsatisfaction,
    e.worklifebalance,
    e.datesurvey,
    m.jobinvolvement,
    m.performancerating,
    m.surveydate,
    r.attrition,
    r.retirementdate,
    r.retirementtype,
    r.resignationreason,
    g.age,
    g.businesstravel,
    g.department,
    g.distancefromhome,
    g.education,
    g.educationfield,
    g.gender,
    g.jobLevel,
    g.jobrole,
    g.maritalStatus,
    g.monthlyIncome,
    g.numcompaniesworked,
    g.percentsalaryhike,
    g.stockoptionlevel,
    g.totalworkingyears,
    g.trainingtimeslastyear,
    g.yearsatcompany,
    g.yearssincelastpromotion,
    g.infodate
FROM employee e
INNER JOIN manager m ON e.employeeid = m.employeeid
INNER JOIN general g ON e.employeeid = g.employeeid
LEFT JOIN retirement r ON e.employeeid = r.employeeid
""", conn)

In [587]:
BD.to_sql("BD",conn,if_exists="replace")

35280

In [588]:
pd.read_sql("""select employeeid,count(*)
                            from BD
                            group by employeeid""", conn)

,employeeid,count(*)
0,1,8
1,2,8
2,3,8
3,4,8
4,5,8
...,...,...
4405,4406,8
4406,4407,8
4407,4408,8
4408,4409,8


In [589]:
pd.read_sql("""select datesurvey,count(*)
                            from employee
                            group by datesurvey""", conn)

,datesurvey,count(*)
0,2015-12-31 00:00:00,4410
1,2016-12-31 00:00:00,4410


In [590]:
pd.read_sql("""select surveydate,count(*)
                            from manager
                            group by surveydate""", conn)

,surveydate,count(*)
0,2015-12-31 00:00:00,4410
1,2016-12-31 00:00:00,4410


In [591]:
pd.read_sql("""SELECT strftime('%Y',retirementdate) as fecha,
                                count(*) as cnt
                                from BD
                                group by fecha""", conn)

,fecha,cnt
0,None,29592
1,2015,488
2,2016,5200


In [592]:
pd.read_sql("""select strftime('%Y', retirementdate) AS FECHA,count(*)
                            from retirement
                            group by FECHA""", conn)

,FECHA,count(*)
0,2015,61
1,2016,650


In [593]:
pd.read_sql("""select strftime('%Y', infodate) AS FECHA,count(*)
                            from general
                            group by FECHA""", conn)

,FECHA,count(*)
0,2015,4410
1,2016,4410


In [594]:
##### 1. Filtrar datos por empleados que tengan evaluación de desempeño en el último año 2016
pd.read_sql("""SELECT *
FROM BD
WHERE strftime('%Y', infodate) = '2016'""", conn)

,index,employeeid,environmentsatisfaction,jobsatisfaction,worklifebalance,datesurvey,jobinvolvement,performancerating,surveydate,attrition,...,maritalstatus,monthlyincome,numcompaniesworked,percentsalaryhike,stockoptionlevel,totalworkingyears,trainingtimeslastyear,yearsatcompany,yearssincelastpromotion,infodate
0,1,1,3.0,4.0,2.0,2015-12-31 00:00:00,3,3,2015-12-31 00:00:00,None,...,Married,131160,1.0,11,0,1.0,6,1,0,2016-12-31 00:00:00
1,3,1,3.0,4.0,2.0,2015-12-31 00:00:00,3,3,2016-12-31 00:00:00,None,...,Married,131160,1.0,11,0,1.0,6,1,0,2016-12-31 00:00:00
2,5,2,3.0,2.0,4.0,2015-12-31 00:00:00,2,4,2015-12-31 00:00:00,Yes,...,Single,41890,0.0,23,1,6.0,3,5,1,2016-12-31 00:00:00
3,7,2,3.0,2.0,4.0,2015-12-31 00:00:00,2,4,2016-12-31 00:00:00,Yes,...,Single,41890,0.0,23,1,6.0,3,5,1,2016-12-31 00:00:00
4,9,3,2.0,2.0,1.0,2015-12-31 00:00:00,3,3,2015-12-31 00:00:00,None,...,Married,193280,1.0,15,3,5.0,2,5,0,2016-12-31 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17635,35271,4408,1.0,3.0,3.0,2016-12-31 00:00:00,3,4,2016-12-31 00:00:00,None,...,Married,37020,0.0,20,0,5.0,4,4,1,2016-12-31 00:00:00
17636,35273,4409,4.0,1.0,3.0,2016-12-31 00:00:00,2,3,2015-12-31 00:00:00,None,...,Divorced,23980,0.0,14,1,10.0,2,9,7,2016-12-31 00:00:00
17637,35275,4409,4.0,1.0,3.0,2016-12-31 00:00:00,2,3,2016-12-31 00:00:00,None,...,Divorced,23980,0.0,14,1,10.0,2,9,7,2016-12-31 00:00:00
17638,35277,4410,1.0,3.0,NaN,2016-12-31 00:00:00,4,3,2015-12-31 00:00:00,None,...,Divorced,54680,0.0,12,0,NaN,6,21,3,2016-12-31 00:00:00


In [595]:
#Se selecciona las renuncias del año 2016 para realizar el modelo
pd.read_sql("""SELECT *
FROM BD
WHERE strftime('%Y', retirementdate) = '2016'""", conn)

,index,employeeid,environmentsatisfaction,jobsatisfaction,worklifebalance,datesurvey,jobinvolvement,performancerating,surveydate,attrition,...,maritalstatus,monthlyincome,numcompaniesworked,percentsalaryhike,stockoptionlevel,totalworkingyears,trainingtimeslastyear,yearsatcompany,yearssincelastpromotion,infodate
0,4,2,3.0,2.0,4.0,2015-12-31 00:00:00,2,4,2015-12-31 00:00:00,Yes,...,Single,41890,0.0,23,1,6.0,3,5,1,2015-12-31 00:00:00
1,5,2,3.0,2.0,4.0,2015-12-31 00:00:00,2,4,2015-12-31 00:00:00,Yes,...,Single,41890,0.0,23,1,6.0,3,5,1,2016-12-31 00:00:00
2,6,2,3.0,2.0,4.0,2015-12-31 00:00:00,2,4,2016-12-31 00:00:00,Yes,...,Single,41890,0.0,23,1,6.0,3,5,1,2015-12-31 00:00:00
3,7,2,3.0,2.0,4.0,2015-12-31 00:00:00,2,4,2016-12-31 00:00:00,Yes,...,Single,41890,0.0,23,1,6.0,3,5,1,2016-12-31 00:00:00
4,24,7,1.0,3.0,1.0,2015-12-31 00:00:00,3,4,2015-12-31 00:00:00,Yes,...,Single,58130,2.0,20,1,5.0,2,0,0,2015-12-31 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,33563,3981,4.0,2.0,4.0,2016-12-31 00:00:00,3,3,2016-12-31 00:00:00,Yes,...,Married,44220,4.0,11,1,13.0,2,9,3,2016-12-31 00:00:00
5196,33616,3995,4.0,1.0,3.0,2016-12-31 00:00:00,3,3,2015-12-31 00:00:00,Yes,...,Married,21680,0.0,14,1,2.0,2,1,0,2015-12-31 00:00:00
5197,33617,3995,4.0,1.0,3.0,2016-12-31 00:00:00,3,3,2015-12-31 00:00:00,Yes,...,Married,21680,0.0,14,1,2.0,2,1,0,2016-12-31 00:00:00
5198,33618,3995,4.0,1.0,3.0,2016-12-31 00:00:00,3,3,2016-12-31 00:00:00,Yes,...,Married,21680,0.0,14,1,2.0,2,1,0,2015-12-31 00:00:00


##Imputación de datos

In [596]:
##as variables que tienen datos nulos son:

#environmentsatisfaction 50
#jobsatisfaction 40
#worklifebalance 76
#numcompaniesworked 38
#totalworkingyears 18
#resignationreason 70

In [597]:
## Se usa la función unique para verificar los datos nulos
print("environmentsatisfaction: ")
print(BD.environmentsatisfaction.unique())
print(" ")
print("jobsatisfaction: ")
print(BD.jobsatisfaction.unique())
print(" ")
print("numcompaniesworked: ")
print(BD.numcompaniesworked.unique())
print(" ")
print("totalworkingyears: ")
print(BD.totalworkingyears.unique())
print(" ")
print("resignationreason: ")
print(BD.resignationreason.unique())
print(" ")
print("worklifebalance: ")
print(BD.worklifebalance.unique())


environmentsatisfaction: 
[ 3.  2.  4.  1. nan]
 
jobsatisfaction: 
[ 4.  2.  1.  3. nan]
 
numcompaniesworked: 
[ 1.  0.  3.  4.  2.  7.  9.  5.  6.  8. nan]
 
totalworkingyears: 
[ 1.  6.  5. 13.  9. 28. 10. 21. 16. 37.  7.  3. 15.  8. nan 12. 17. 19.
 22.  2.  4. 23.  0. 11. 24. 25. 20. 14. 26. 18. 30. 36. 31. 33. 32. 34.
 40. 29. 35. 27. 38.]
 
resignationreason: 
[None 'Others' 'Stress' 'Salary']
 
worklifebalance: 
[ 2.  4.  1.  3. nan]


In [598]:
## verificamos que tipo de variable es
BD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35280 entries, 0 to 35279
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   employeeid               35280 non-null  int64  
 1   environmentsatisfaction  35080 non-null  float64
 2   jobsatisfaction          35120 non-null  float64
 3   worklifebalance          34976 non-null  float64
 4   datesurvey               35280 non-null  object 
 5   jobinvolvement           35280 non-null  int64  
 6   performancerating        35280 non-null  int64  
 7   surveydate               35280 non-null  object 
 8   attrition                5688 non-null   object 
 9   retirementdate           5688 non-null   object 
 10  retirementtype           5688 non-null   object 
 11  resignationreason        5128 non-null   object 
 12  age                      35280 non-null  int64  
 13  businesstravel           35280 non-null  object 
 14  department            

In [599]:
### Con la función creada tratamos los datos nulos de la base de detos que son de tipo númerico
BD = funciones.impute_columns(df = BD, columns = ['environmentsatisfaction', 'jobsatisfaction', 'worklifebalance', 'totalworkingyears'], strategy = 'median')

In [600]:
#En este caso los cambiamos a CERO, ya que hay personas que pueden estar en su primer empleo y todavia no cumplen 1 año.
BD['numcompaniesworked'] = BD['numcompaniesworked'].fillna(0)
BD['attrition'] = BD['attrition'].fillna('No')
#Para este caso, a pesr de que la media y la mediana estan en 11.27 y 10, lo cambiaremos por el total de años que lleva en la empresa 'Years at company'
BD['totalworkingyears'].fillna(BD['yearsatcompany'], inplace=True)

In [601]:
BD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35280 entries, 0 to 35279
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   employeeid               35280 non-null  int64  
 1   environmentsatisfaction  35280 non-null  float64
 2   jobsatisfaction          35280 non-null  float64
 3   worklifebalance          35280 non-null  float64
 4   datesurvey               35280 non-null  object 
 5   jobinvolvement           35280 non-null  int64  
 6   performancerating        35280 non-null  int64  
 7   surveydate               35280 non-null  object 
 8   attrition                35280 non-null  object 
 9   retirementdate           5688 non-null   object 
 10  retirementtype           5688 non-null   object 
 11  resignationreason        5128 non-null   object 
 12  age                      35280 non-null  int64  
 13  businesstravel           35280 non-null  object 
 14  department            

In [602]:
BD['attrition'].unique()

array(['No', 'Yes'], dtype=object)

In [603]:
BD.to_csv('BD.csv', index=False)
